In [61]:
#Loading the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as web
import scipy.optimize as sco

In [62]:
# Getting stock data
companies = ['AMZN', 'AAPL', 'NFLX', 'GOOG']
#price_data = web.get_data_yahoo(companies,start = '2014-01-01', end = '2018-05-31')['Adj Close']
price_data = web.get_data_yahoo(companies,start = '2014-01-01', end = '2014-12-30')['Adj Close']
price_data

Symbols,AMZN,AAPL,NFLX,GOOG
Date,,,,
2014-01-02,397.970001,17.516609,51.831429,554.481689
2014-01-03,396.440002,17.131842,51.871429,550.436829
2014-01-06,393.630005,17.225262,51.367142,556.573853
2014-01-07,398.029999,17.102074,48.500000,567.303589
2014-01-08,401.920013,17.210373,48.712856,568.484192
...,...,...,...,...
2014-12-23,306.290009,25.472557,48.061428,529.137268
2014-12-24,303.029999,25.352591,48.871429,527.322266
2014-12-26,309.089996,25.800751,48.578571,532.567810


In [64]:
# Getting the log returns
log_ret = np.log(price_data/price_data.shift(1))
log_ret

Symbols,AMZN,AAPL,NFLX,GOOG
Date,,,,
2014-01-02,NaN,NaN,NaN,NaN
2014-01-03,-0.003852,-0.022211,0.000771,-0.007322
2014-01-06,-0.007113,0.005438,-0.009769,0.011088
2014-01-07,0.011116,-0.007177,-0.057435,0.019095
2014-01-08,0.009726,0.006313,0.004379,0.002079
...,...,...,...,...
2014-12-23,-0.000816,-0.003548,-0.000743,0.010839
2014-12-24,-0.010701,-0.004721,0.016713,-0.003436
2014-12-26,0.019801,0.017523,-0.006010,0.009898


In [65]:
num_rows, num_cols = price_data.shape

In [66]:
equal_weights = np.array(
  [1 / num_cols] * num_cols
)
equal_weights

array([0.25, 0.25, 0.25, 0.25])

In [157]:
######################################################## Minimum Variance ####################################################
def minimum_variance(weights,covariance):
    return np.sqrt(np.transpose(weights) @ (covariance * 253) @ weights)

In [158]:
V = np.array([[5, 9, 1, 3], [ 1, 7, 8, 3], [1,6, 11,1], [1, 3, 8, 4]])
minimum_variance(equal_weights,V)

33.741665637605976

In [159]:
bounds = tuple((0, 1) for w in equal_weights) #We again constrain the weights such that all weights fall within the interval :
cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x)-1.0}) #sum of weights equals to one
sco.minimize(minimum_variance, equal_weights, args=(V), method='SLSQP',bounds = bounds, constraints=cons) 

     fun: 28.44545911955863
     jac: array([28.44573474, 34.53481579, 28.44449902, 28.44529557])
 message: 'Optimization terminated successfully'
    nfev: 38
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([4.05411241e-01, 9.47666023e-14, 1.80045450e-02, 5.76584214e-01])

In [161]:
##################################################### Maximum Diversification #################################################
def calc_diversification_ratio(w, V, vol):
    diversification_ratio= (equal_weights.T @ np.diag(V) / (np.sqrt((equal_weights.T @ V @ equal_weights))))
    # return negative for minimization problem (maximize = minimize -)
    return -diversification_ratio

In [171]:
vol=minimum_variance(equal_weights,V)
calc_diversification_ratio(equal_weights, V, vol)

-3.1819805153394642

In [162]:
sco.minimize(calc_diversification_ratio, equal_weights, args=(V,vol), method='SLSQP',bounds = bounds, constraints=cons) 

     fun: -3.1819805153394642
     jac: array([0., 0., 0., 0.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([0.25, 0.25, 0.25, 0.25])

In [163]:
############################################################ Risk Parity ######################################################

def calculate_risk_contribution(w,V):
    # function that calculates asset contribution to total risk
    #w = np.matrix(w)
    sigma = minimum_variance(w,V)
    # Marginal Risk Contribution
    MRC = V @ w.T
    # Risk Contribution
    RC = np.multiply(MRC,w.T)/sigma
    return RC

def risk_budget_objective(w,V):
    x_t = w # risk target in percent of portfolio risk
    sig_p =   minimum_variance(x_t,V) # portfolio sigma
    risk_target = np.asmatrix(np.multiply(sig_p,x_t))
    asset_RC = calculate_risk_contribution(x_t,V)
    J = sum(np.square(asset_RC-risk_target.T))[0,0] # sum of squared error
    return J

In [166]:
risk_budget_objective(equal_weights,V)

282.37944664031613

In [167]:
sco.minimize(risk_budget_objective, equal_weights, args=(V), method='SLSQP',bounds = bounds, constraints=cons) 

     fun: 264.50243438934285
     jac: array([1060.73516464, 1060.73441315, 1060.73930359, 1060.73278046])
 message: 'Optimization terminated successfully'
    nfev: 43
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.31274181, 0.18869509, 0.18837117, 0.31019192])

In [178]:
#####################################################################
#               PORTFOLIO Optimization functions                    #
#####################################################################

def optimization(V, vol, nameFunction, w0=equal_weights):
    # V: covariance matrix
    # bnd: individual position limit
    # long only: long only constraint
    # w0: initial weight: equal_weights
    bounds = tuple((0, 1) for w in equal_weights) #We again constrain the weights such that all weights fall within the interval :
    cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x)-1.0}) #sum of weights equals to one
    if nameFunction == "Minimum variance":
        results = sco.minimize(minimum_variance, w0, args=(V), method='SLSQP',bounds = bounds, constraints=cons)
                  
    elif nameFunction == "Maximum diversification":
        results = sco.minimize(calc_diversification_ratio, w0, args=(V,vol), method='SLSQP',bounds = bounds, constraints=cons)
    
    else:
        results = sco.minimize(risk_budget_objective, w0, args=(V), method='SLSQP',bounds = bounds, constraints=cons)
    return results

In [179]:
optimization(V, vol, "Minimum variance")

     fun: 28.44545911955863
     jac: array([28.44573474, 34.53481579, 28.44449902, 28.44529557])
 message: 'Optimization terminated successfully'
    nfev: 38
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([4.05411241e-01, 9.47666023e-14, 1.80045450e-02, 5.76584214e-01])

In [180]:
optimization(V, vol, "Maximum diversification")

     fun: -3.1819805153394642
     jac: array([0., 0., 0., 0.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([0.25, 0.25, 0.25, 0.25])

In [181]:
optimization(V, vol, "Risk Parity")

     fun: 264.50243438934285
     jac: array([1060.73516464, 1060.73441315, 1060.73930359, 1060.73278046])
 message: 'Optimization terminated successfully'
    nfev: 43
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.31274181, 0.18869509, 0.18837117, 0.31019192])

In [197]:
#####################################################################
#               UNCONDITIONAL APPROACH USING ROLLING WINDOWS        #
#####################################################################
def unconditional(method):
    #Rolling windows parameteres
    M=20 #windows size
    H=10 #shift

    #initilisation
    L=0
    count=0
    num_rows, num_cols = price_data.shape
    
    #Rolling windows
    while (M +H*count) <= num_rows:
        #updating Rolling windows parameteres
        L = M +H*count
        K = H*count

        print("Rolling Window: [",K,L,"]")

        # Portfolio Returns on window
        lg_returns = log_ret.iloc[K:L] #log returns for each window

        #uncondtional covariance for each window
        cov_mat = lg_returns.cov() * 252
        cov_mat= cov_mat.to_numpy()
        
        if  method == "Minimum variance": 
            #################################################### Minimum variance ###################################################
            print("Minimum Variance")
            results = optimization(cov_mat, 0,"Minimum variance")
            
        elif method == "Maximum diversification":
        #################################################### Maximum Diversification ####################################################
            print("Maximum Diversification")
            vol=  minimum_variance(equal_weights,cov_mat)
            results = optimization(cov_mat, vol,"Maximum diversification")
            
        else:
            #################################################### Risk Parity ###################################################
            print("Risk Parity")
            results = optimization(cov_mat, 0,"Risk Parity")

        print("Poids optimaux", results.x,)
        
        sd = (minimum_variance(results.x,cov_mat))
        port_ret = np.sum(lg_returns.mean() * results.x)
        port_ret = (port_ret + 1) ** 252 - 1  #annualize

        sharpe_ratio = port_ret / sd
        print("Sharpe Ratio",sharpe_ratio)
        print(" ")

        count= count + 1

In [198]:
unconditional("Minimum variance")

Rolling Window: [ 0 20 ]
Minimum Variance
Poids optimaux [0.14058349 0.30184363 0.0972232  0.46034967]
Sharpe Ratio -0.038778364572337
 
Rolling Window: [ 10 30 ]
Minimum Variance
Poids optimaux [0.12030632 0.37309118 0.08625469 0.42034781]
Sharpe Ratio 0.10407777623830503
 
Rolling Window: [ 20 40 ]
Minimum Variance
Poids optimaux [0.0819656  0.58430702 0.         0.33372738]
Sharpe Ratio 0.36035501251271457
 
Rolling Window: [ 30 50 ]
Minimum Variance
Poids optimaux [0.19794374 0.33178055 0.         0.47027571]
Sharpe Ratio -0.10822264654771736
 
Rolling Window: [ 40 60 ]
Minimum Variance
Poids optimaux [3.61505058e-02 7.93495715e-01 1.70353779e-01 2.63482558e-16]
Sharpe Ratio -0.1791018189303018
 
Rolling Window: [ 50 70 ]
Minimum Variance
Poids optimaux [2.03999696e-16 9.35011844e-01 6.49881564e-02 3.68656468e-16]
Sharpe Ratio -0.1281994963271956
 
Rolling Window: [ 60 80 ]
Minimum Variance
Poids optimaux [7.70325644e-17 5.23735938e-01 6.32469578e-02 4.13017104e-01]
Sharpe Ratio 0.

In [200]:
unconditional("Maximum diversification")

Rolling Window: [ 0 20 ]
Maximum Diversification
Poids optimaux [0.25 0.25 0.25 0.25]
Sharpe Ratio 0.04158410675114695
 
Rolling Window: [ 10 30 ]
Maximum Diversification
Poids optimaux [0.25 0.25 0.25 0.25]
Sharpe Ratio 0.22293115012811154
 
Rolling Window: [ 20 40 ]
Maximum Diversification
Poids optimaux [0.25 0.25 0.25 0.25]
Sharpe Ratio 0.14535522083883312
 
Rolling Window: [ 30 50 ]
Maximum Diversification
Poids optimaux [0.25 0.25 0.25 0.25]
Sharpe Ratio -0.06698675136804501
 
Rolling Window: [ 40 60 ]
Maximum Diversification
Poids optimaux [0.25 0.25 0.25 0.25]
Sharpe Ratio -0.28878190974493895
 
Rolling Window: [ 50 70 ]
Maximum Diversification
Poids optimaux [0.25 0.25 0.25 0.25]
Sharpe Ratio -0.20009985014794385
 
Rolling Window: [ 60 80 ]
Maximum Diversification
Poids optimaux [0.25 0.25 0.25 0.25]
Sharpe Ratio -0.10152745405646915
 
Rolling Window: [ 70 90 ]
Maximum Diversification
Poids optimaux [0.25 0.25 0.25 0.25]
Sharpe Ratio 0.14207596571729864
 
Rolling Window: [ 80 

In [201]:
unconditional("Risk Parity")

Rolling Window: [ 0 20 ]
Risk Parity
Poids optimaux [0.26987757 0.29839814 0.1389937  0.29273059]
Sharpe Ratio -0.025141530780855778
 
Rolling Window: [ 10 30 ]
Risk Parity
Poids optimaux [0.18595266 0.33738722 0.15267011 0.32399002]
Sharpe Ratio 0.14220609129860903
 
Rolling Window: [ 20 40 ]
Risk Parity
Poids optimaux [0.14285283 0.36560505 0.174868   0.31667413]
Sharpe Ratio 0.2934779636095826
 
Rolling Window: [ 30 50 ]
Risk Parity
Poids optimaux [0.27089773 0.31962628 0.0816439  0.3278321 ]
Sharpe Ratio -0.06887646766205492
 
Rolling Window: [ 40 60 ]
Risk Parity
Poids optimaux [0.16916156 0.49048111 0.16016544 0.1801919 ]
Sharpe Ratio -0.2714144077387104
 
Rolling Window: [ 50 70 ]
Risk Parity
Poids optimaux [0.18932648 0.50095534 0.13835688 0.17136129]
Sharpe Ratio -0.21543178679955682
 
Rolling Window: [ 60 80 ]
Risk Parity
Poids optimaux [0.16582177 0.36387894 0.16872571 0.30157358]
Sharpe Ratio -0.07010645095161683
 
Rolling Window: [ 70 90 ]
Risk Parity
Poids optimaux [0.105